In [ ]:
# 04_demo.ipynb

import cv2
import math
from ultralytics import YOLO

# ---------------------------------------------------------
# 1. CONFIGURATION
# ---------------------------------------------------------
# PATH TO YOUR MODEL (Ensure best.pt is inside 'models/trained/')
MODEL_PATH = '../models/trained/best.pt'

# CHECK YOUR DATASET FOR CORRECT CLASS NAMES
# 0 is usually the first class in your alphabetized Roboflow list
# Update this list to match YOUR data.yaml exactly
classNames = ['Hardhat', 'NO-Hardhat', 'Person']

# Settings
COLOR_SAFE = (0, 255, 0)    # Green
COLOR_DANGER = (0, 0, 255)  # Red
CONF_THRESHOLD = 0.5        # 50% confidence required

# ---------------------------------------------------------
# 2. LOAD MODEL
# ---------------------------------------------------------
try:
    model = YOLO(MODEL_PATH)
    print("Model loaded successfully!")
except Exception as e:
    print("Error loading model. Did you download best.pt and put it in models/trained/?")
    print(e)

# ---------------------------------------------------------
# 3. HELPER FUNCTION
# ---------------------------------------------------------
def get_color(cls_name):
    if cls_name == "NO-Hardhat": # DANGER CLASS
        return COLOR_DANGER
    elif cls_name == "Hardhat":  # SAFE CLASS
        return COLOR_SAFE
    return (255, 255, 0) # Yellow for generic 'Person'

# ---------------------------------------------------------
# 4. LIVE WEBCAM LOOP
# ---------------------------------------------------------
# Use '0' for default webcam, '1' for external
cap = cv2.VideoCapture(0)
cap.set(3, 1280) # Width
cap.set(4, 720)  # Height

print("Starting Live Stream... Press 'q' to quit.")

while True:
    success, img = cap.read()
    if not success:
        break

    # Inference
    results = model(img, stream=True, verbose=False)

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100

            if conf > CONF_THRESHOLD:
                # Bounding Box
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                # Class
                cls = int(box.cls[0])
                current_class = classNames[cls] if cls < len(classNames) else "Unknown"

                # Styling
                color = get_color(current_class)

                # Draw
                cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)

                # Label
                label = f'{current_class} {conf}'
                t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                c2 = x1 + t_size[0], y1 - t_size[1] - 3
                cv2.rectangle(img, (x1, y1), c2, color, -1, cv2.LINE_AA)  # Filled header
                cv2.putText(img, label, (x1, y1 - 2), 0, 1, [255, 255, 255], thickness=2, lineType=cv2.LINE_AA)

                # ALERT LOGIC
                if current_class == "NO-Hardhat":
                    cv2.putText(img, "SAFETY VIOLATION!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_DANGER, 3)

    cv2.imshow('Smart Helmet Detector', img)

    # Exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()